![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [36]:
import quantconnect
from utils import aggregate, concat_dfs, get_start_price
from technical_analysis import get_poc
# from strategies import measure_period_profit
# from strategies import add_expirations

In [37]:
qbw = quantconnect.QuantBookWrapper({'qb':QuantBook(),'Resolution':Resolution,'OptionRight':OptionRight})

In [38]:
qb = QuantBook()
tsla = qb.AddEquity("TSLA")
equity_symbol = tsla.Symbol

In [39]:
tsla = qbw.get_tsla(3000)

In [41]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime

In [42]:
def get_available(start=(2022, 8, 25), right_abrev='c'):
    if isinstance(start, tuple):
        start = datetime(*start)
    contract_symbols = qb.OptionChainProvider.GetOptionContractList(equity_symbol, start)
    if right_abrev == 'c':
        right = OptionRight.Call
    else:
        right = OptionRight.Put

    options = [(s.ID.Date, s.ID.StrikePrice) for s in contract_symbols if s.ID.OptionRight == right]
    df = pd.DataFrame(data=options, columns=['expiry', 'strike'])
    df['days_since_start'] = df['expiry'] - start
    df = df.sort_values('strike').sort_values('days_since_start')

    return df

In [ ]:
def get_start_price(df, g, expiration):
    options_create = g.underlying_open.first()
    df = df.merge(options_create, left_on=['year' ,expiration], right_on=['year' ,expiration], suffixes=('', '_b'))
    df = df.rename(columns={'underlying_open_b': 'start_price'})
    return df

In [ ]:
def add_expirations(df, expiration):
    df['date'] = df.index if is_datetime(df.index) else pd.to_datetime(df.index)
    df = df.reset_index(drop=True)
    df['week'] = df['date'].dt.week
    df['year'] = df['date'].dt.year
    if expiration == 'month':
        df['month'] = df['week'] // 4
    g = df.groupby(['year', expiration])
    options_exp = g.date.last()
    options_exp.iloc[-1] += timedelta(days=4 - options_exp.iloc[-1].weekday())  # make final expiry a friday
    df = df.merge(options_exp, left_on=['year', expiration], right_on=['year', expiration], suffixes=('', '_expiration'))
    df['dte'] = (df['date_expiration'] - df['date'])/pd.Timedelta(1.0, unit='D')

    return df, g

In [56]:
def measure_period_profit(df, strategy, expiration='week', update_freq='candle', poc_window=0):
    df = df.rename(columns={'open': 'underlying_open', 'high': 'underlying_high',
                        'low': 'underlying_low', 'close': 'underlying_close'})
    df['strategy_open'] = 0
    df['strategy_close'] = 0
    df['hourly_profit'] = 0
    df, g = add_expirations(df, expiration=expiration)
    
    if update_freq == 'candle':
        if poc_window:
            df = concat_dfs(df, get_poc(df, poc_window))
            guide = 'poc'
        else:
            guide = 'underlying_open'
    elif update_freq == 'once':
        df = concat_dfs(df, get_start_price(df, g, expiration))
        guide = 'start_price'

    df = strategy.get_strikes(df, guide)
    legs = [l.split('_strike')[0] for l in strategy.legs]
    for leg in legs:
        df[leg+'_open'] = 0
        df[leg+'_close'] = 0
    
    # print(df.head())
    for ih, (date, candle) in enumerate(df.iterrows()):
        # df.at[ih, 'strategy_open'], df.at[ih, 'strategy_close'] = strategy.candle_profit(candle)
        legs_open, legs_close = strategy.candle_profit(candle, combine_legs=False)
        df.at[ih, 'strategy_open'], df.at[ih, 'strategy_close'] = np.sum(legs_open),  np.sum(legs_close)
        for leg, open, close in zip(legs, legs_open, legs_close):
            df.at[ih, leg+'_open'] = open
            df.at[ih, leg+'_close'] = close

    df['hourly_profit'] = -df['strategy_close'].diff()
    # df['hourly_profit'][df['new_option']] = df['strategy_open'] - df['strategy_close']
    df.loc[df.new_option.array, 'hourly_profit'] = df['strategy_open'] - df['strategy_close']
    df['running_profit'] = df['hourly_profit'].cumsum()

    return df


In [44]:
def option_history(strike, expiry, start=(2022, 8, 25), right_abrev='c', res_abrev='h', split_correct=(2022, 8, 25)):
    if split_correct:
        if start < datetime(*split_correct):
            strike *= 3
    expiry = expiry.replace(hour=0, minute=0)
    start = start.replace(hour=0, minute=0)
    contract_symbols = qb.OptionChainProvider.GetOptionContractList(equity_symbol, start)
    if right_abrev == 'c':
        right =  OptionRight.Call
    else:
        right =  OptionRight.Put
    if res_abrev == 'h':
        resolution = Resolution.Hour
    else:
        raise NotImplementedError

    options = [s for s in contract_symbols if s.ID.OptionRight == right and s.ID.StrikePrice == strike and s.ID.Date == expiry]
    if len(options) == 0:
        print('empty options', f'strike: {strike}, expiry: {expiry}, start: {start}, right_abrev: {right_abrev}, res_abrev: {res_abrev}')
        # start_tuple = attrgetter(*('year', 'month', 'day'))(start)
        df = get_available(start)
        df = df[df['expiry'] == datetime(2022,2,18)]
        print('all options at that expiry: ', df)
    assert len(options) == 1
    history = qb.History(options[0], start, expiry + timedelta(days=1), resolution)
    return history

In [45]:
class IronCondors():
    def __init__(self, long_offset=5, short_offset=5, wing_distance=1, use_historical=True):
        self.long_offset = long_offset
        self.short_offset = short_offset
        self.wing_distance = wing_distance
        self.legs = ['sell_call_strike', 'buy_call_strike', 'sell_put_strike', 'buy_put_strike']
        self.use_historical = use_historical
        self.option_history = {}

    def get_strikes(self, df, guide):
        df['sell_call_strike'] = df[guide] * (1 + self.short_offset / 100.)
        df['buy_call_strike'] = df['sell_call_strike'] * (1 + self.wing_distance / 100.)
        df['sell_put_strike'] = df[guide] * (1 - self.long_offset / 100.)
        df['buy_put_strike'] = df['sell_put_strike'] * (1 - self.wing_distance / 100.)

        if self.use_historical:
            for index, row in df.iterrows():
                for i, leg in enumerate(self.legs):
                    meta = leg.split('_')
                    contract = meta[1][0]
                    if i % 2 == 0:
                        strikes = get_available_strikes(
                            row['date'], 
                            row['date_expiration'], contract
                        )
                        if row['date_expiration'] == datetime(2022,2,18):
                            print(row['date'], strikes)
                        if len(strikes) == 0:  # use previous in the case of misssing data
                            strikes = np.array([df.iloc[index-1][leg]])
                        
                    
                    df.at[index, leg] = strikes[np.argmin(np.abs(strikes-row[leg]))]
        
        df['new_option'] = (df.sell_call_strike.diff() + df.date_expiration.diff()/pd.Timedelta(1.0, unit='D')) != 0.  #+ df.right.diff()
        return df

    def candle_profit(self, candle, combine_legs=True):
        if combine_legs:
            open, close = 0, 0
        else:
            open, close = [], []
        for leg in self.legs:
            meta = leg.split('_')
            contract = meta[1][0]
            money_gained = [-1, 1][meta[0] == 'sell']
            if self.use_historical:
                if candle['new_option']:
                    self.option_history[leg] = option_history(candle[leg], candle['date_expiration'], 
                                                            start=candle['date'], right_abrev=contract).droplevel([0,1,2,3])
                    # print(candle['date'], leg.split('_strike')[0], self.option_history[leg][self.option_history[leg].index == candle['date']][['open', 'close']])
                leg_open = self.option_history[leg][self.option_history[leg].index == candle['date']]['open'].array[0]
                leg_close = self.option_history[leg][self.option_history[leg].index == candle['date']]['close'].array[0]
            else:
                leg_open = op.black_scholes(
                    K=candle[leg], St=candle['underlying_open'], r=3, t=candle['dte']+1./24, v=53, type=contract
                )['value']['option value']
                leg_close = op.black_scholes(
                    K=candle[leg], St=candle['underlying_close'], r=3, t=candle['dte'], v=53, type=contract
                )['value']['option value']
            leg_open *= money_gained 
            leg_close *= money_gained
            if not combine_legs:
                leg_open, leg_close = [leg_open], [leg_close]
            open += leg_open
            close += leg_close
            # print('open', leg_open, open, np.sum(open))
            
        return open, close

In [46]:
def get_available_strikes(start=(2022, 8, 25), expiration=(2022,10,14), right_abrev='c', split_correct=(2022, 8, 25)):
    # start = datetime(*start)
    # expiration = datetime(*expiration)
    expiration = expiration.replace(hour=0, minute=0)
    if start.replace(hour=0, minute=0) == expiration:
        start -= timedelta(days=1)
    contract_symbols = qb.OptionChainProvider.GetOptionContractList(equity_symbol, start)
    if right_abrev == 'c':
        right = OptionRight.Call
    else:
        right = OptionRight.Put

    strikes = np.array([s.ID.StrikePrice for s in contract_symbols if s.ID.OptionRight == right and s.ID.Date == expiration])
    if split_correct:
        if start < datetime(*split_correct):
            strikes /= 3

    return strikes

In [62]:
ic.loc[40:42][['underlying_close', 'underlying_open', 'strategy_open', 'strategy_close',	'sell_call_open', 'sell_call_close','buy_call_open','buy_call_close','sell_put_open','sell_put_close','buy_put_open','buy_put_close']]

In [66]:
option_history(345.0, datetime(2022,4,8), start=datetime(2022,4,8), right_abrev='p').droplevel([0,1,2,3])

In [42]:
ic.columns

In [45]:
ic[['underlying_open', 'sell_call_strike', 'sell_put_strike', 'sell_call_open', 'sell_put_open']]

In [26]:
ic.iloc[30:80]

In [47]:
from analysis import movement_vs_profit
import plots

In [45]:
for offsets in [7.5,15]:
    ic_strat = IronCondors(long_offset=offsets, short_offset=offsets)
    ic = measure_period_profit(tsla,  
        ic_strat,
        expiration='week',
        update_freq='once')
    # print(ic.head())
    movement_vs_profit(ic)
    plots.plot_candles_and_profit(ic, lines=ic_strat.legs)

In [54]:
ic_strat = IronCondors(long_offset=7.5, short_offset=7.5)
ic = measure_period_profit(tsla,  
    ic_strat,
    expiration='week',
    update_freq='once')

In [55]:
ic

In [58]:
df = ic
strategy = ic_strat
legs = [l.split('_strike')[0] for l in strategy.legs]
for leg in legs:
    df[leg+'_open'] = 0
    df[leg+'_close'] = 0

# print(df.head())
for ih, (date, candle) in enumerate(df.iterrows()):
    # df.at[ih, 'strategy_open'], df.at[ih, 'strategy_close'] = strategy.candle_profit(candle)
    legs_open, legs_close = strategy.candle_profit(candle, combine_legs=False)
    df.at[ih, 'strategy_open'], df.at[ih, 'strategy_close'] = np.sum(legs_open),  np.sum(legs_close)
    for leg, open, close in zip(legs, legs_open, legs_close):
        df.at[ih, leg+'_open'] = open
        df.at[ih, leg+'_close'] = close

In [59]:
df['hourly_profit'] = -df['strategy_close'].diff()
# df['hourly_profit'][df['new_option']] = df['strategy_open'] - df['strategy_close']
df.loc[df.new_option.array, 'hourly_profit'] = df['strategy_open'] - df['strategy_close']
df['running_profit'] = df['hourly_profit'].cumsum()

In [60]:
ic = df

In [61]:
movement_vs_profit(ic)
plots.plot_candles_and_profit(ic, lines=ic_strat.legs)

In [62]:
for offsets in [10,15]:
    ic_strat = IronCondors(long_offset=offsets, short_offset=offsets)
    ic = measure_period_profit(tsla,  
        ic_strat,
        expiration='week',
        update_freq='once')
    # print(ic.head())
    movement_vs_profit(ic)
    plots.plot_candles_and_profit(ic, lines=ic_strat.legs)

In [63]:
for offsets in [20]:
    ic_strat = IronCondors(long_offset=offsets, short_offset=offsets)
    ic = measure_period_profit(tsla,  
        ic_strat,
        expiration='week',
        update_freq='once')
    # print(ic.head())
    movement_vs_profit(ic)
    plots.plot_candles_and_profit(ic, lines=ic_strat.legs)

In [65]:
ic.iloc[200:250]

In [69]:
ic.loc[350:400][[
    'underlying_close', 'underlying_open', 'strategy_open', 'strategy_close',	
    #'sell_call_open', 'sell_call_close','buy_call_open','buy_call_close', 'sell_put_open','sell_put_close','buy_put_open','buy_put_close', 
    'date', 'dte',
    'running_profit'
    ]]

In [70]:
ic.iloc[371]

In [76]:
option_history(185., datetime(2021,5,7), start=datetime(2021, 5, 3), right_abrev='p').iloc[:5]

In [77]:
option_history(190., datetime(2021,5,7), start=datetime(2021, 5, 3), right_abrev='p').iloc[:5]

In [78]:
option_history(180., datetime(2021,5,7), start=datetime(2021, 5, 3), right_abrev='p').iloc[:5]

In [81]:
plots.plot_candles_and_profit(ic, lines=ic_strat.legs, metric='strategy_open')

In [25]:
ic.iloc[30:80]

In [21]:
plots.plot_candles_and_profit(ic, lines=ic_strat.legs, show_afterhours=True)

In [22]:
import plots

In [23]:
plots.plot_candles_and_profit(ic, lines=ic_strat.legs)

In [24]:
ic.iloc[75:100]

In [ ]:
ic_strat = IronCondors()
ic = measure_period_profit(tsla,  
    ic_strat,
    expiration='month',
    update_freq='once')
ic
movement_vs_profit(ic)
plots.plot_candles_and_profit(ic, lines=ic_strat.legs)

In [27]:
df =get_available((2022,3,23))

In [36]:
df.sort_values('days_since_start')[100:150]

In [31]:
plots.scatter_heatmap(df['strike'].array, df['days_since_start'].dt.days.array)

In [37]:
df, g = add_expirations(tsla, expiration='month')

In [44]:
def add_expirations(df, expiration):
    df['date'] = df.index if is_datetime(df.index) else pd.to_datetime(df.index)
    df = df.reset_index(drop=True)
    df['week'] = df['date'].dt.week
    if expiration == 'month':
        df['month'] = df['week'] // 4
    g = df.groupby(expiration)
    options_exp = g.date.last()
    options_exp.iloc[-1] += timedelta(days=4 - options_exp.iloc[-1].weekday())  # make final expiry a friday
    df = df.merge(options_exp, left_on=expiration, right_on=expiration, suffixes=('', '_expiration'))
    df['dte'] = (df['date_expiration'] - df['date'])/pd.Timedelta(1.0, unit='D')

    return df, g

In [42]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime

In [45]:
df, g = add_expirations(tsla, expiration='month')
df[:50]

In [46]:
ic_strat = IronCondors()
ic = measure_period_profit(tsla,  
    ic_strat,
    expiration='month',
    update_freq='once')
ic
movement_vs_profit(ic)
plots.plot_candles_and_profit(ic, lines=ic_strat.legs, show_afterhours=False)

In [47]:
plots.plot_candles_and_profit(ic, lines=ic_strat.legs, show_afterhours=False)

In [48]:
ic_strat = IronCondors(20,20)
ic = measure_period_profit(tsla,  
    ic_strat,
    expiration='month',
    update_freq='once')
ic
movement_vs_profit(ic)
plots.plot_candles_and_profit(ic, lines=ic_strat.legs, show_afterhours=False)

In [49]:
ic_strat = IronCondors(10,10, 5)
ic = measure_period_profit(tsla,  
    ic_strat,
    expiration='month',
    update_freq='once')
ic
movement_vs_profit(ic)
plots.plot_candles_and_profit(ic, lines=ic_strat.legs, show_afterhours=False)

In [51]:
ic_strat = IronCondors(10,10, 5)
ic = measure_period_profit(tsla,  
    ic_strat,
    expiration='month',
    update_freq='candle')
ic
movement_vs_profit(ic)
plots.plot_candles_and_profit(ic, lines=ic_strat.legs, show_afterhours=False)

In [62]:
df = get_available((2022,9,28))
df.sort_values('days_since_start')[:50].sort_values('strike')

In [52]:
ic_strat = IronCondors(10,10, 5)
ic = measure_period_profit(tsla[:800],  
    ic_strat,
    expiration='month',
    update_freq='candle')
ic
movement_vs_profit(ic)
plots.plot_candles_and_profit(ic, lines=ic_strat.legs, show_afterhours=False)

In [63]:
ic_strat = IronCondors(10,10, 5)
ic = measure_period_profit(tsla[:800],  
    ic_strat,
    expiration='month',
    update_freq='candle',
    poc_window=30)
ic
movement_vs_profit(ic)
plots.plot_candles_and_profit(ic, lines=ic_strat.legs, show_afterhours=False)